### 피아노 Midi 사이트
- https://www.freepianotutorials.net/2023/12/ludwig-goransson-can-you-hear-music.html#more

### Ⅰ. 라이브러리

In [2]:
import os
import mido
import numpy as np
import pandas as pd

from mido import MidiFile, MidiTrack, MetaMessage, Message

In [3]:
data_folder = "midi_data"

In [4]:
def load_midi_data(input_name, target_name):
    input_path  = os.path.join(data_folder, input_name)
    target_path = os.path.join(data_folder, target_name)
    
    input_mid  = mido.MidiFile(input_path)
    target_mid = mido.MidiFile(target_path)
    
    return input_mid, target_mid

In [5]:
#input_name = "Fugue3.mid"
input_name = "Fugue3.mid"
target_name = "Fugue1.mid"

input_mid, target_mid = load_midi_data(input_name, target_name)

### Ⅱ. Data Extraction

#### 1. note (음정) : [ 0 to 127 ]
- msg.type = 'note_on' / 'note_off'
    + 소리의 강도 / 세기 ( velocity )
    + 지속 시간 ( time, 단위 : tick )
    
- note_on / note_off 구분 X
    + note_on channel=0 note=66 velocity=64 time=0
    + note_off channel=0 note=66 velocity=48 time=60
        - note_off 도 (( note / velocity )) 존재

In [7]:
def distribute_msg(mid_track):
    track_dict = {}
    
    track_dict['MetaMessage'] = []
    track_dict['Message'] = []
    track_dict['ETC'] = []
    
    for msg in mid_track:
        
        if isinstance(msg, MetaMessage):
            track_dict['MetaMessage'].append(msg)
            
        elif isinstance(msg, Message):
            track_dict['Message'].append(msg)
            
        else:
            track_dict['ETC'].append(msg)
            
    return track_dict

In [8]:
def extract_note(track_file, mid_msg):
    
    def initialize_note_ticks():
        tick_type = []
        tick_note = []
        tick_chan = []
        tick_velo = []
    
        return tick_type, tick_note, tick_chan, tick_velo
    
    df_note_col = {'tick', 'count', 'msg_type', 'msg_note', 'msg_velocity', 'msg_channel'}
    df_note = pd.DataFrame(columns = df_note_col)
    
    note_cur_time = 0
    note_start = True
    note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
    
    def note_check(msg_type):
        if msg_type !='note_on' and msg_type !='note_off':
            return False
        
        else:
            return True
    
    def insert_note_data(note_tick_type, note_tick_note, note_tick_chan, note_tick_velo):
        until_tick = df_note.shape[0]
                
        for note_idx in range(until_tick, note_cur_time):
            df_note.loc[note_idx, "tick"] = note_cur_time
            df_note.loc[note_idx, "count"] = len(note_tick_note)
            df_note.loc[note_idx, "msg_type"] = note_tick_type
            df_note.loc[note_idx, "msg_note"] = note_tick_note
            df_note.loc[note_idx, "msg_velocity"] = note_tick_velo
            df_note.loc[note_idx, "msg_channel"] = note_tick_chan
            
    
    for msg in mid_msg:
        
        # note_check
        if note_check(msg.type) == False:
            continue
        
        # note_type
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_typ = 'note_off'
        else:
            msg_typ = msg.type
            
        if msg.time > 0:
            
            if note_start == True:
                note_start = False
            else:
                insert_note_data(note_tick_type, note_tick_note, note_tick_chan, note_tick_velo)
                note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
                
            note_cur_time = msg.time + note_cur_time
            
        note_tick_type.append(msg_typ)
        note_tick_note.append(msg.note)
        note_tick_chan.append(msg.channel)
        note_tick_velo.append(msg.velocity)
            
    if len(note_tick_note) > 0:
        insert_note_data(note_tick_type, note_tick_note, note_tick_chan, note_tick_velo)

    df_note.to_csv(track_file)
    
    return df_note

In [ ]:
def extract_data_info(file_name, mid):
    mid_info  = {}
    mid_track = []
    
    # mid_info
    mid_info['type'] = mid.type
    mid_info['ticks_per_beat'] = mid.ticks_per_beat
    mid_info['track'] = mid_track
    
    for idx, track in enumerate(mid.tracks):
        
        track_dict = distribute_msg(track)
        
        track_file_name = file_name.split(".")[0] + "_" + str(idx) + ".csv"
        df_note = extract_note(track_file_name, track_dict['Message'])
        
        print(df_note)
        mid_track.append(df_note)
        
    return mid_info

input_info = extract_data_info(input_name, input_mid)
target_info = extract_data_info(target_name, target_mid)

- 비교 알고리즘 다시 생각해볼 필요가 있음
    + time 기준
    + 어떤 것을 바탕으로 평가 알고리즘

In [272]:
def processing_note_on_data(input_track, target_track):
    input_notes = []
    target_notes = []
    for note_track in input_track:
        for index in range(note_track.shape[0]):
            for count in range(note_track.loc[index,'count']):
                if note_track.loc[index, 'msg_type'][count] == 'note_on':
                    input_notes.append(note_track.loc[index, 'msg_note'][count])
            
    for note_track in target_track:
        for index in range(note_track.shape[0]):
            for count in range(note_track.loc[index,'count']):
                if note_track.loc[index, 'msg_type'][count] == 'note_on':
                    target_notes.append(note_track.loc[index, 'msg_note'][count])
                
    return input_notes, target_notes

In [273]:
def calculate_note_accuracy(input_info, target_info, limit_offset=0):
    """
    인풋 MIDI와 정답 MIDI의 pitch 정확도 계산 - Yeong-Min Ko
    
    Args:
        input_midi: 인풋 MIDI 파일
        target_midi: 타겟 MIDI 파일
        
    Return:
        pitch 정확도
    """

    # 인풋 MIDI와 타겟 MIDI의 음표 정보를 추출
    input_notes, target_notes = processing_note_on_data(input_info['track'], target_info['track'])
    
    # 피치 정확도 계산
    # time 별로 기준을 세울 필요가 있음
    total_accuracy = 0
    for input_note in input_notes:
        for reference_note in target_notes:
            if abs(input_note - reference_note) < limit_offset:
                total_accuracy += 1
                break

    # 전체 음표 수와 정확한 음표 수를 이용하여 정확도 계산
    pitch_accuracy = (total_accuracy / len(target_notes)) * 100
    if pitch_accuracy > 100:
        pitch_accuracy %= 100
    
    return pitch_accuracy


In [274]:
def calculate_accuracy_level(accuracy):
    """
    음정 정확도에 따라 Level을 계산하는 함수

    Args:
        accuracy: 음정 정확도

    Return:
        계산된 Level
    """
    level = int(min(10, round(accuracy / 10)))  # 10% 단위로 Level 계산, 최대 Level은 10, 반올림 적용
    return level

In [275]:
note_accuracy = calculate_note_accuracy(input_info, target_info, 5)
note_level = calculate_accuracy_level(note_accuracy)

In [276]:
print(f'Pitch Accuracy testCase 1: {note_accuracy:.2f}% - Level {note_level}')
print(note_level)

Pitch Accuracy testCase 1: 21.23% - Level 2
2


In [270]:
input_info['track'][1].shape[0]

52800

In [1]:
target_info['track'][1]

NameError: name 'target_info' is not defined

In [ ]:
for index in range(target_info['track'][1].shape[0],input_info['track'][1].shape[0] ):
    

In [284]:
input_df = input_info['track'][1]
target_df = target_info['track'][1]

limit_offset = 5
total_accuracy = 0

for tick in range(input_df.shape[0]):
    for input_count in range(input_df.loc[tick, 'count']):
        if input_df.loc[tick, 'msg_type'][input_count] == 'note_on':
            input_note = input_df.loc[tick, 'msg_note'][input_count]
            
            if tick > target_df.shape[0]:
                target_note = 0
                if abs(input_note - target_note) < limit_offset:
                    total_accuracy += 1
                continue
                
            for target_count in range(target_df.loc[tick, 'count']):
                target_note = target_df.loc[tick, 'msg_note'][count]
                if abs(input_note - target_note) < limit_offset:
                    total_accuracy += 1
                    break
                
pitch_accuracy = (total_accuracy / len(target_notes)) * 100
if pitch_accuracy > 100:
    pitch_accuracy %= 100
    
print(total_accuracy)
print(pitch_accuracy)

IndexError: list index out of range

#### 8. 옥타브 ( note )

#### 전제 조건
- track
    + 길이가 다르면 처음부터 측정 불가능
    + track 은 **왼손 / 오른손 2개**만 있다는 가정

In [ ]:
def pre_check(input_mid, target_mid):
    # track 길이 비교
    input_track_len  = len(input_mid.tracks)
    target_track_len = len(target_mid.tracks)
    
    #return input_track_len == target_track_len
    #### time 비교하는 것 짜기
    return True

### Ⅲ. Data Comparision

In [90]:
print(len(input_mid.tracks))
print(len(target_mid.tracks))

4
5


In [93]:
for idx, track in enumerate(target_mid.tracks):
    print(f"Track {idx + 1}: {track.name}")

Track 1: 
Track 2: 
Track 3: 
Track 4: 
Track 5: 
